# 02. Model Scratch

Workspace for testing models on the fly


### Import Data


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import auc
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn import svm

RANDOM_STATE = 42

In [ ]:
data_df = None
if os.path.isfile("data/data.pkl"):
    data_df = pd.read_pickle("data/data.pkl")
else:
    print("Load Data")
data_df.head()

### Test TBI Classification Alone


In [ ]:
# tbi_df = data_df[data_df["Study"] == "TBI"]
# tbi_evc = np.vstack(tbi_df["EVC"])
# tbi_class = np.where(tbi_df["Diagnosis"] == "POS", 1, 0)
# print(np.shape(tbi_evc))
# tbi_df.head()

tbi_df = data_df[(data_df["Study"] == "TBI") & (data_df["Harmonized"].notna())]
tbi_evc = np.vstack(tbi_df["EVC"])
tbi_class = np.where(tbi_df["Diagnosis"] == "POS", 1, 0)
print(np.shape(tbi_evc))
tbi_df.sample(5)

In [ ]:
# X is tbi_evc and Y is tbi_class
# clf = svm.SVC(
#     kernel="sigmoid",
#     random_state=42,
#     class_weight="balanced",
#     probability=True,
#     gamma="scale",
# ) 2, 7, 83, 86, 120, 167

from ipywidgets import IntProgress
from IPython.display import display
import time

best_roc_auc = 0.00
state = 1
f = IntProgress(min=0, max=1000)

while state < 200:
    clf = svm.SVC(
        kernel="sigmoid", class_weight="balanced", probability=True, random_state=state
    )
    loo = LeaveOneOut()

    results = []
    for train, test in loo.split(tbi_evc, tbi_class):
        this_fit = clf.fit(tbi_evc[train], tbi_class[train])
        results.append(
            [
                tbi_class[test],
                this_fit.predict_proba(tbi_evc[test])[:, 1],
                this_fit.predict(tbi_evc[test]),
            ]
        )
    results = np.squeeze(results)

    # ROC_AUC
    fpr, tpr, thresholds = roc_curve(results[:, 0], results[:, 1])
    roc_auc = auc(fpr, tpr)
    print(roc_auc)

    # Confusion Matrix
    x_train, x_test, y_train, y_test = train_test_split(
        tbi_evc, tbi_class, random_state=state, stratify=tbi_class
    )
    holdout_test = clf.fit(x_train, y_train)

    y_pred = holdout_test.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)

    if roc_auc > best_roc_auc:
        best_roc_auc = roc_auc
        best_roc_auc_data = (fpr, tpr, cm)

    state = state + 1
    f.value += 1

# Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 6))

ax1.plot(
    best_roc_auc_data[0],
    best_roc_auc_data[1],
    lw=2,
    label=f"LOOCV ROC (AUC = {best_roc_auc:.2f})",
)
ax1.plot(
    [0, 1], [0, 1], linestyle="--", lw=2, color="k", label="Chance level", alpha=0.8
)
ax1.set_xlim([-0.05, 1.05])
ax1.set_ylim([-0.05, 1.05])
ax1.set_xlabel("FPR")
ax1.set_ylabel("TPR")
ax1.set_title("ROC")
ax1.legend(loc="lower right")
ax1.grid(True)
ax1.set_aspect("equal", adjustable="box")

disp = ConfusionMatrixDisplay(
    confusion_matrix=best_roc_auc_data[2], display_labels=clf.classes_
)
disp.plot(ax=ax2, cmap=plt.cm.Blues, colorbar=False)
ax2.set_title("Holdout Confusion Matrix")
plt.show()

print(classification_report(tbi_class, results[:, 2]))

In [ ]:
# Hyperparameter Optimization Test

# x_train, x_test, y_train, y_test = train_test_split(tbi_evc, tbi_class, random_state=42)

# svc_params = {
#         'C': np.linspace(1e-7, 10, 10),
#         'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
#         'tol': np.linspace(1e-5, 0.1, 5),
#         'gamma': np.linspace(1e-5, 10, 10),
#         'coef0': np.linspace(1e-2, 2, 3)
#     }

# gs_results = GridSearchCV(svm.SVC(random_state=42), svc_params, cv=LeaveOneOut(), verbose=True).fit(x_train, y_train)
# results = pd.DataFrame(gs_results.cv_results_)

# results = results.loc[:, ('rank_test_score', 'mean_test_score', 'params')]
# results.sort_values(by='rank_test_score', ascending=True, inplace=True)

# score = results['mean_test_score'].iloc[0]
# params = results['params'].iloc[0]

# print(f'Best score for GridSearchCv is {score:.3f}')
# print(f'Params: {params}')

### Auto ML


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
# Expands EVC column, copied from 03_Per_Node_Model.ipynb
df = data_df.drop(
    columns=[
        "Data",
        "Connectome",
        "Harmonized",
        "CLU",
        "DIV",
        "Site",
        "Subject_ID",
        "Study",
    ]
)
expanded_columns = []

for idx, row in df.iterrows():
    evc_matrix = row["EVC"]
    if evc_matrix is not None:
        expanded_columns.append(evc_matrix.flatten())
    else:
        expanded_columns.append([np.nan] * 271)

evc_expanded = pd.DataFrame(expanded_columns, index=df.index)
evc_expanded.columns = [f"EVC_{i}" for i in range(evc_expanded.shape[1])]

df = pd.concat([df.drop(columns=["EVC"]), evc_expanded], axis=1)


def get_cohort(d):
    return "AD" if d in ["AD", "HC"] else "TBI"


def get_diag_status(d):
    return "Case" if d in ["AD", "POS"] else "Control"


df["Cohort"] = df["Diagnosis"].apply(get_cohort)
df["DiagStatus"] = df["Diagnosis"].apply(get_diag_status)
df["Diagnosis"] = df["Diagnosis"].apply(lambda x: 1 if x in ["AD", "POS"] else 0)
df["Sex"] = df["Sex"].map({"M": 1, "m": 1, "F": 0, "f": 0})
df["Cohort"] = pd.Categorical(df["Cohort"])
df["DiagStatus"] = pd.Categorical(df["DiagStatus"])
df = df.dropna(subset=["EVC_0"])
df = df.drop(columns=["DiagStatus", "Sex", "Age"])

tbi_df = df[df["Cohort"] == "TBI"]
ad_df = df[df["Cohort"] == "AD"]
tbi_df.sample(5)

In [ ]:
# tbi_df = tbi_df.loc[
#     :,
#     df.columns.intersection(
#         ["Diagnosis", "EVC_1", "EVC_4", "EVC_77", "EVC_80", "EVC_114", "EVC_160"]
#     ),
# ]
autogl_ad_predictor = TabularPredictor(
    label="Diagnosis", problem_type="binary", log_to_file=True
).fit(TabularDataset(tbi_df), presets="best")